In [ ]:
import pandas as pd

df=pd.read_csv('/content/gdrive/My Drive/ISY5002/Project/Image_Search/data/car_make model_2.csv')
df['CarModel']= df['CarModel'].apply(lambda x: x.lower())
corpus = df['CarModel'].astype(str).values.tolist()

#put the search terms, and corpus
base_document = corpus


#threhold
highest_score = 1
lowest_score = 0.55

#top result
top_k = 5


In [ ]:
#ref: https://towardsdatascience.com/machine-learning-in-production-keras-flask-docker-and-heroku-933b5f885459

from flask import Flask, render_template, request
from flask_uploads import UploadSet, configure_uploads, IMAGES

from keras.preprocessing.image import load_img
from model import process_image, predict_class

from keywordsearch import cosine_similarities_test, get_bert_similarity


app = Flask(__name__)
photos= UploadSet('photos', IMAGES)

app.config['UPLOADED_PHOTOS_DEST']='./static/img'
configure_uploads(app,photos)

@app.route('/home', methods=['GET','POST'])
def home():
  welcome = "Hello, World !"
  return welcome

# the main route for upload and prediction
@app.route('/upload', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST' and 'photo' in request.files:
        # save the image
        filename = photos.save(request.files['photo'])
        # load the image
        image = load_img('./static/img/'+filename, target_size=(2000, 2000))
        # process the image
        image = process_image(image)
        # make prediction
        prediction = predict_class(image)
        recommend_carmodels=get_bert_similarity(prediction, base_document)
        # the answer which will be rendered back to the user
        pic='./static/img/'+filename

        return '''
        <h1><img src="{}" height = "387" width="473"/><h1>
        <h1>The classification is: {} </h1>
        <h2>More Results:<br>
        {}<br>
        {}<br>
        {}<br>
        {}<br>
        {}<br>
        <h2>

        
        '''.format(pic,prediction, recommend_carmodels[0],recommend_carmodels[1],recommend_carmodels[2],recommend_carmodels[3],recommend_carmodels[4] )

    # web page to show before the POST request containing the image
    return render_template('upload.html')




if __name__ == '__main__':
    app.run(host='0.0.0.0', debug=True)